# 期末程式作業I

姓名：涂育賢  
系級：經濟四  
學號：105208077  

## 實作word to vector

### 導入套件

In [1]:
import tensorflow as tf
import math
import numpy as np

### word2vec模型架構

In [2]:
def __is_bounded(direction, range, index, tokens_leng):
    cover = range*direction
    if cover+index < 0 or cover+index >= tokens_leng:
        return True
    else:
        return False


def get_context(tokens, window_size):
    context_pair = []
    for i, token in enumerate(tokens):
        for j in range(1, window_size+1):
            if not __is_bounded(1, j, i, len(tokens)):
                context_pair.append((tokens[i], tokens[i+j]))
            if not __is_bounded(-1, j, i, len(tokens)):
                context_pair.append((tokens[i], tokens[i-j]))
    return context_pair


def __get_word_set(tokens):
    word_set = set()
    for token in tokens:
        word_set.add(token)
    return word_set


def __get_word_index(word_set):
    word_index_dic = dict()
    inverse_word_dic = dict()
    for i, word in enumerate(word_set):
        word_index_dic[word] = i
        inverse_word_dic[i] = word
    return word_index_dic, inverse_word_dic


def generate_batch(context_pair, batch_size):
    batch_list = []
    batch = []
    for i, pair in enumerate(context_pair):

        if i % batch_size == 0 and i != 0:
            batch_list.append(batch)
            batch = []

        batch.append(pair)
    return batch_list


def get_vec(word, session):
    return session.run(embeddings[word_index_dic[word]])


def __dis(vec1, vec2):
    dis = 0.0
    for i in range(0, len(vec1)):
        dis += math.pow((vec1[i]-vec2[i]), 2)
    return dis


def get_cos_similarity(vec1, vec2):
    vec1_leng = 0
    for value in vec1:
        vec1_leng += (value*value)
    vec1_leng = math.sqrt(vec1_leng)
    vec2_leng = 0
    for value in vec2:
        vec2_leng += (value*value)
    vec2_leng = math.sqrt(vec2_leng)
    product = np.dot(vec1, vec2)

    return product/(vec1_leng*vec2_leng)


def __sim(vec1, vec2):
    return (1 - math.acos(get_cos_similarity(vec1, vec2)) / math.pi)


def one_hot(data, label_size):
    vector = np.zeros((len(data), label_size), dtype='f')
    for i, single in enumerate(data):
        vector[i][single] = 1.0
    return vector


def find_cloest_word(word_set, session, target_word):
    sim = 0.0
    vec1 = get_vec(target_word, session)
    result = ''
    for word in word_set:
        if word == target_word:
            continue
        vec2 = get_vec(word, session)
        tmp_sim = __sim(vec1, vec2)
        print('%s : %s : %s' % (target_word, word, tmp_sim))
        if tmp_sim > sim:
            sim = tmp_sim
            result = word
    return result

### 丟入想要看的句子，來觀察字向量

In [3]:
text = ["It's time to sleep", "Having enough sleep can help me grow stronger", 'I can sleep for twenty hours']

In [4]:
window_size = 2  # 觀看前後文時會看距離兩格的詞
embedding_size = 5  # 學習到的vector長度為5 >>> 前2詞+ (該詞) +前2詞

### 訓練

In [5]:
import tensorflow as tf
context_pair = []
word_set = set()

for sentence in text:
    tokens = sentence.lower().split(' ')
    context_pair += get_context(tokens, window_size)
    tmp_word_set = __get_word_set(tokens)
    for word in tmp_word_set:
        word_set.add(word)

word_index_dic, inverse_word_dic = __get_word_index(word_set)
word_size = len(word_set)
batch_size = len(context_pair)
inputs = [word_index_dic[x[0]] for x in context_pair]
labels = [word_index_dic[x[1]] for x in context_pair]

train_inputs = tf.placeholder(
    tf.int32, shape=[batch_size])  # placeholder是資料放置的地方
train_labels = tf.placeholder(tf.int32, shape=[batch_size, ])

embeddings = tf.Variable(
    tf.random_uniform([word_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

nce_weights = tf.Variable(
    tf.truncated_normal([word_size, embedding_size],
                        stddev=1.0 / math.sqrt(embedding_size)))

nce_biases = tf.Variable(tf.zeros([word_size]))

prediction = tf.add(
    tf.matmul(embed, tf.transpose(nce_weights)), nce_biases)
train_labels_vector = tf.one_hot(train_labels, word_size)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=prediction, labels=train_labels_vector))
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=1.0).minimize(loss)
session = tf.Session()
init = tf.global_variables_initializer()
session.run(init)
for iteration in range(0, 10000):
    total_loss = 0

    feed_dict = {train_inputs: inputs, train_labels: labels}
    _, cur_loss, pred = session.run(
        [optimizer, loss, prediction], feed_dict=feed_dict)
    print('%s: loss: %s' % (iteration, cur_loss))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0: loss: 2.7509527
1: loss: 2.6968458
2: loss: 2.6494253
3: loss: 2.6078496
4: loss: 2.5713565
5: loss: 2.5392454
6: loss: 2.5108664
7: loss: 2.4856257
8: loss: 2.4629848
9: loss: 2.4424677
10: loss: 2.423662
11: loss: 2.4062173
12: loss: 2.3898425
13: loss: 2.3742995
14: loss: 2.3593967
15: loss: 2.344982
16: loss: 2.3309357
17: loss: 2.317165
18: loss: 2.3035986
19: loss: 2.2901824
20: loss: 2.2768755
21: loss: 2.263647
22: loss: 2.250476
23: loss: 2.2373464
24: loss: 2.2242486
25: loss: 2.2111762
26: loss: 2.198126
27: loss: 2.1850972
28: loss: 2.1720915
29: loss: 2.1591113
30: loss: 2.1461594
31: loss: 2.1332402
32: loss: 2.1203585
33: loss: 2.1075184
34: loss: 2.094725
35: loss: 2.0819821
36: loss: 2.0692947
37: loss: 2.0566666
38: loss: 2.0441017
39: loss: 2.0316038
40: loss: 2.0191762
41: lo

457: loss: 1.3790501
458: loss: 1.3790158
459: loss: 1.3789818
460: loss: 1.378948
461: loss: 1.3789145
462: loss: 1.378881
463: loss: 1.3788476
464: loss: 1.3788146
465: loss: 1.3787819
466: loss: 1.3787491
467: loss: 1.3787168
468: loss: 1.3786848
469: loss: 1.3786527
470: loss: 1.3786207
471: loss: 1.378589
472: loss: 1.3785577
473: loss: 1.3785263
474: loss: 1.3784952
475: loss: 1.3784643
476: loss: 1.3784337
477: loss: 1.3784032
478: loss: 1.3783727
479: loss: 1.3783425
480: loss: 1.3783122
481: loss: 1.3782827
482: loss: 1.378253
483: loss: 1.3782234
484: loss: 1.378194
485: loss: 1.3781646
486: loss: 1.3781358
487: loss: 1.3781068
488: loss: 1.3780783
489: loss: 1.3780497
490: loss: 1.3780214
491: loss: 1.3779932
492: loss: 1.3779651
493: loss: 1.3779372
494: loss: 1.3779095
495: loss: 1.3778819
496: loss: 1.3778543
497: loss: 1.3778272
498: loss: 1.3778001
499: loss: 1.3777732
500: loss: 1.3777463
501: loss: 1.3777196
502: loss: 1.3776932
503: loss: 1.3776666
504: loss: 1.37764

941: loss: 1.3725095
942: loss: 1.3725045
943: loss: 1.3724991
944: loss: 1.3724942
945: loss: 1.3724891
946: loss: 1.372484
947: loss: 1.3724787
948: loss: 1.3724738
949: loss: 1.3724687
950: loss: 1.3724638
951: loss: 1.3724588
952: loss: 1.3724538
953: loss: 1.3724488
954: loss: 1.3724437
955: loss: 1.3724388
956: loss: 1.372434
957: loss: 1.3724289
958: loss: 1.3724241
959: loss: 1.372419
960: loss: 1.3724142
961: loss: 1.3724093
962: loss: 1.3724045
963: loss: 1.3723997
964: loss: 1.3723948
965: loss: 1.3723899
966: loss: 1.3723853
967: loss: 1.3723803
968: loss: 1.3723755
969: loss: 1.3723708
970: loss: 1.3723658
971: loss: 1.3723611
972: loss: 1.3723564
973: loss: 1.3723518
974: loss: 1.3723471
975: loss: 1.3723421
976: loss: 1.3723376
977: loss: 1.3723328
978: loss: 1.3723282
979: loss: 1.3723235
980: loss: 1.3723189
981: loss: 1.3723142
982: loss: 1.3723096
983: loss: 1.372305
984: loss: 1.3723005
985: loss: 1.3722959
986: loss: 1.3722913
987: loss: 1.3722864
988: loss: 1.3722

1414: loss: 1.3710222
1415: loss: 1.3710203
1416: loss: 1.3710184
1417: loss: 1.3710165
1418: loss: 1.3710146
1419: loss: 1.3710128
1420: loss: 1.3710107
1421: loss: 1.3710089
1422: loss: 1.371007
1423: loss: 1.3710052
1424: loss: 1.3710032
1425: loss: 1.3710014
1426: loss: 1.3709995
1427: loss: 1.3709975
1428: loss: 1.3709959
1429: loss: 1.3709937
1430: loss: 1.3709921
1431: loss: 1.3709902
1432: loss: 1.3709882
1433: loss: 1.3709865
1434: loss: 1.3709847
1435: loss: 1.3709826
1436: loss: 1.370981
1437: loss: 1.3709792
1438: loss: 1.3709772
1439: loss: 1.3709755
1440: loss: 1.3709737
1441: loss: 1.3709717
1442: loss: 1.3709699
1443: loss: 1.3709681
1444: loss: 1.3709663
1445: loss: 1.3709645
1446: loss: 1.3709626
1447: loss: 1.3709608
1448: loss: 1.3709592
1449: loss: 1.3709571
1450: loss: 1.3709555
1451: loss: 1.3709536
1452: loss: 1.3709519
1453: loss: 1.3709501
1454: loss: 1.3709482
1455: loss: 1.3709465
1456: loss: 1.3709447
1457: loss: 1.370943
1458: loss: 1.3709413
1459: loss: 1

1891: loss: 1.3703738
1892: loss: 1.3703729
1893: loss: 1.370372
1894: loss: 1.370371
1895: loss: 1.37037
1896: loss: 1.370369
1897: loss: 1.370368
1898: loss: 1.3703672
1899: loss: 1.370366
1900: loss: 1.3703653
1901: loss: 1.3703642
1902: loss: 1.3703634
1903: loss: 1.3703622
1904: loss: 1.3703614
1905: loss: 1.3703604
1906: loss: 1.3703593
1907: loss: 1.3703585
1908: loss: 1.3703575
1909: loss: 1.3703567
1910: loss: 1.3703557
1911: loss: 1.3703547
1912: loss: 1.3703538
1913: loss: 1.370353
1914: loss: 1.370352
1915: loss: 1.3703508
1916: loss: 1.37035
1917: loss: 1.3703492
1918: loss: 1.3703482
1919: loss: 1.3703471
1920: loss: 1.3703465
1921: loss: 1.3703455
1922: loss: 1.3703445
1923: loss: 1.3703437
1924: loss: 1.3703427
1925: loss: 1.3703417
1926: loss: 1.3703408
1927: loss: 1.3703399
1928: loss: 1.3703389
1929: loss: 1.3703378
1930: loss: 1.370337
1931: loss: 1.370336
1932: loss: 1.3703353
1933: loss: 1.3703345
1934: loss: 1.3703336
1935: loss: 1.3703326
1936: loss: 1.3703316
1

...

8198: loss: 1.3691608
8199: loss: 1.3691608
8200: loss: 1.369161
8201: loss: 1.3691608
8202: loss: 1.3691608
8203: loss: 1.3691607
8204: loss: 1.3691607
8205: loss: 1.3691605
8206: loss: 1.3691605
8207: loss: 1.3691605
8208: loss: 1.3691605
8209: loss: 1.3691605
8210: loss: 1.3691605
8211: loss: 1.3691605
8212: loss: 1.3691604
8213: loss: 1.3691604
8214: loss: 1.3691603
8215: loss: 1.3691603
8216: loss: 1.3691603
8217: loss: 1.3691603
8218: loss: 1.3691603
8219: loss: 1.3691603
8220: loss: 1.3691603
8221: loss: 1.3691602
8222: loss: 1.3691602
8223: loss: 1.3691598
8224: loss: 1.3691599
8225: loss: 1.3691598
8226: loss: 1.3691599
8227: loss: 1.3691599
8228: loss: 1.3691599
8229: loss: 1.3691598
8230: loss: 1.3691597
8231: loss: 1.3691597
8232: loss: 1.3691596
8233: loss: 1.3691596
8234: loss: 1.3691596
8235: loss: 1.3691596
8236: loss: 1.3691596
8237: loss: 1.3691596
8238: loss: 1.3691596
8239: loss: 1.3691595
8240: loss: 1.3691592
8241: loss: 1.3691592
8242: loss: 1.3691592
8243: loss:

8611: loss: 1.3691467
8612: loss: 1.3691467
8613: loss: 1.3691468
8614: loss: 1.3691466
8615: loss: 1.3691467
8616: loss: 1.3691466
8617: loss: 1.3691467
8618: loss: 1.3691466
8619: loss: 1.3691466
8620: loss: 1.3691466
8621: loss: 1.3691465
8622: loss: 1.3691466
8623: loss: 1.3691462
8624: loss: 1.3691465
8625: loss: 1.3691462
8626: loss: 1.3691462
8627: loss: 1.3691462
8628: loss: 1.3691462
8629: loss: 1.3691461
8630: loss: 1.3691462
8631: loss: 1.3691462
8632: loss: 1.3691461
8633: loss: 1.3691461
8634: loss: 1.369146
8635: loss: 1.3691461
8636: loss: 1.369146
8637: loss: 1.369146
8638: loss: 1.3691458
8639: loss: 1.3691458
8640: loss: 1.3691458
8641: loss: 1.3691459
8642: loss: 1.3691455
8643: loss: 1.3691455
8644: loss: 1.3691458
8645: loss: 1.3691455
8646: loss: 1.3691455
8647: loss: 1.3691455
8648: loss: 1.3691455
8649: loss: 1.3691455
8650: loss: 1.3691455
8651: loss: 1.3691455
8652: loss: 1.3691454
8653: loss: 1.3691453
8654: loss: 1.3691454
8655: loss: 1.3691454
8656: loss: 1

9025: loss: 1.3691341
9026: loss: 1.369134
9027: loss: 1.369134
9028: loss: 1.3691338
9029: loss: 1.3691338
9030: loss: 1.3691338
9031: loss: 1.3691338
9032: loss: 1.3691338
9033: loss: 1.3691338
9034: loss: 1.3691338
9035: loss: 1.3691337
9036: loss: 1.3691337
9037: loss: 1.3691337
9038: loss: 1.3691336
9039: loss: 1.3691336
9040: loss: 1.3691336
9041: loss: 1.3691336
9042: loss: 1.3691336
9043: loss: 1.3691335
9044: loss: 1.3691336
9045: loss: 1.3691332
9046: loss: 1.3691336
9047: loss: 1.3691335
9048: loss: 1.3691335
9049: loss: 1.3691332
9050: loss: 1.3691331
9051: loss: 1.3691332
9052: loss: 1.3691332
9053: loss: 1.3691331
9054: loss: 1.3691331
9055: loss: 1.3691331
9056: loss: 1.3691331
9057: loss: 1.3691331
9058: loss: 1.369133
9059: loss: 1.369133
9060: loss: 1.3691329
9061: loss: 1.3691329
9062: loss: 1.3691329
9063: loss: 1.3691329
9064: loss: 1.3691329
9065: loss: 1.3691329
9066: loss: 1.3691329
9067: loss: 1.3691325
9068: loss: 1.3691329
9069: loss: 1.3691325
9070: loss: 1.

9445: loss: 1.3691224
9446: loss: 1.3691224
9447: loss: 1.3691223
9448: loss: 1.3691223
9449: loss: 1.3691223
9450: loss: 1.3691223
9451: loss: 1.3691221
9452: loss: 1.3691221
9453: loss: 1.369122
9454: loss: 1.369122
9455: loss: 1.369122
9456: loss: 1.369122
9457: loss: 1.369122
9458: loss: 1.369122
9459: loss: 1.3691218
9460: loss: 1.3691218
9461: loss: 1.3691218
9462: loss: 1.3691218
9463: loss: 1.3691217
9464: loss: 1.3691217
9465: loss: 1.3691217
9466: loss: 1.3691217
9467: loss: 1.3691217
9468: loss: 1.3691217
9469: loss: 1.3691217
9470: loss: 1.3691216
9471: loss: 1.3691214
9472: loss: 1.3691216
9473: loss: 1.3691216
9474: loss: 1.3691216
9475: loss: 1.3691214
9476: loss: 1.3691214
9477: loss: 1.3691214
9478: loss: 1.3691216
9479: loss: 1.3691214
9480: loss: 1.3691214
9481: loss: 1.3691214
9482: loss: 1.3691213
9483: loss: 1.3691214
9484: loss: 1.3691213
9485: loss: 1.3691211
9486: loss: 1.3691211
9487: loss: 1.3691211
9488: loss: 1.3691211
9489: loss: 1.3691211
9490: loss: 1.36

9824: loss: 1.3691125
9825: loss: 1.3691125
9826: loss: 1.3691125
9827: loss: 1.3691125
9828: loss: 1.3691125
9829: loss: 1.3691125
9830: loss: 1.3691125
9831: loss: 1.3691124
9832: loss: 1.3691124
9833: loss: 1.3691124
9834: loss: 1.3691124
9835: loss: 1.3691123
9836: loss: 1.3691125
9837: loss: 1.3691124
9838: loss: 1.3691124
9839: loss: 1.3691123
9840: loss: 1.3691123
9841: loss: 1.3691123
9842: loss: 1.3691123
9843: loss: 1.3691123
9844: loss: 1.3691123
9845: loss: 1.3691123
9846: loss: 1.3691121
9847: loss: 1.3691121
9848: loss: 1.3691121
9849: loss: 1.3691121
9850: loss: 1.3691118
9851: loss: 1.3691119
9852: loss: 1.3691118
9853: loss: 1.3691119
9854: loss: 1.3691118
9855: loss: 1.3691118
9856: loss: 1.3691118
9857: loss: 1.3691118
9858: loss: 1.3691118
9859: loss: 1.3691118
9860: loss: 1.3691118
9861: loss: 1.3691118
9862: loss: 1.3691117
9863: loss: 1.3691117
9864: loss: 1.3691117
9865: loss: 1.3691115
9866: loss: 1.3691115
9867: loss: 1.3691115
9868: loss: 1.3691115
9869: loss

### 觀察結果

In [7]:
print(find_cloest_word(word_set, session, 'sleep'))

sleep : hours : 0.5920284402038101
sleep : can : 0.4200598447400431
sleep : me : 0.6222435157365194
sleep : twenty : 0.5118854919007744
sleep : enough : 0.3559729083439733
sleep : stronger : 0.3670874456849791
sleep : it's : 0.6059596564272303
sleep : to : 0.48714839223442774
sleep : grow : 0.388255029783524
sleep : help : 0.3020503051943316
sleep : for : 0.4404986160075961
sleep : time : 0.3905636297534706
sleep : i : 0.3572517569620485
sleep : having : 0.45283724665165404
me


In [8]:
print(find_cloest_word(word_set, session, 'stronger'))

stronger : hours : 0.5379878616156762
stronger : can : 0.612115053993628
stronger : me : 0.6388633094029526
stronger : twenty : 0.4532480091393094
stronger : enough : 0.4578428141745903
stronger : it's : 0.26860152004342597
stronger : to : 0.21981287253952508
stronger : grow : 0.6470128331446102
stronger : help : 0.787946753675263
stronger : for : 0.5653396156933479
stronger : time : 0.32126513246643074
stronger : i : 0.5493909907346806
stronger : having : 0.5145491949683716
stronger : sleep : 0.3670874456849791
help


In [9]:
print(find_cloest_word(word_set, session, 'twenty'))

twenty : hours : 0.6551219538822486
twenty : can : 0.599336515111101
twenty : me : 0.242481368907692
twenty : enough : 0.43439160956203826
twenty : stronger : 0.4532480091393094
twenty : it's : 0.3590456973725713
twenty : to : 0.6037206481136248
twenty : grow : 0.22647780600586143
twenty : help : 0.45717673429663386
twenty : for : 0.6844939053497114
twenty : time : 0.4974625677189106
twenty : i : 0.5227360951227857
twenty : having : 0.64096134922433
twenty : sleep : 0.5118854919007744
for
